In [1]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import json
import re
from dotenv import load_dotenv
import os
import requests
load_dotenv()

c:\Users\jaide\Documents\CodingProjects\NeuroSymbolicAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
GCP_API_KEY = os.getenv("GOOGLE_CLOUD_API_KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH_ENGINE_ID")
base_url = "https://www.googleapis.com/customsearch/v1?"

In [13]:
params = {
    "key": GCP_API_KEY,
    "cx": SEARCH_ENGINE_ID,
    "q": "Who is the current president of the United States?"
}
res = requests.get(base_url, params)

In [30]:
data = res.json()
results = data["items"]
results[0]

{'kind': 'customsearch#result',
 'title': 'Joe Biden: The President | The White House',
 'htmlTitle': 'Joe Biden: The <b>President</b> | The White House',
 'link': 'https://www.whitehouse.gov/administration/president-biden/',
 'displayLink': 'www.whitehouse.gov',
 'snippet': 'Joe Biden · A Leader in the Senate and 47th Vice President of the United States · The 46th President of the United States.',
 'htmlSnippet': 'Joe Biden &middot; A Leader in the Senate and 47th Vice <b>President of the United States</b> &middot; The 46th <b>President of the United States</b>.',
 'formattedUrl': 'https://www.whitehouse.gov/administration/president-biden/',
 'htmlFormattedUrl': 'https://www.whitehouse.gov/administration/<b>president</b>-biden/',
 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQq6rGaMCsIJkQNNuVM-ziWDQwQTJHxXiExdGvI8pUAsPg1twBiIiOFoWI&s',
    'width': '259',
    'height': '194'}],
  'metatags': [{'application-name': 'The White House',
    '

In [31]:
model_name = "bartowski/Mistral-7B-Instruct-v0.3-GGUF"
model_file = "Mistral-7B-Instruct-v0.3-Q4_K_M.gguf"
model_path = hf_hub_download(model_name, filename=model_file)
model = Llama(
    model_path = model_path,
    n_ctx=32000,
    verbose=False
)

In [41]:
search_prompt = """
You are a helpful assistant with only 1 job:
1) Search the internet

Always includes a THOUGHT, and a RESPONSE. Afterwards, return the PAUSE keyword.

Whenever asked a question, break down the question step by step. Figure out what information you need to answer the question.

###EXAMPLE 1:
User: Who is the current president of the United States?

THOUGHT: This is a simple question, I just need to look up who the current president of the United States is.
RESPONSE: current president of the United States
PAUSE
###END EXAMPLE 1

###EXAMPLE 2:
User: What is the largest animal in the world?

THOUGHT: I can simply look up the largest animal in the world
RESPONSE: largest animal in the world
PAUSE
###END EXAMPLE 2
"""
answer_prompt = """
You are a helpful assistant with access to the provided context below. You may or may not need to use the provided context.

Do not directly restate the context provided, but rather, rephrase the context into your own words and explain the context to the user.
"""

In [33]:
query = "How much is Elon Musk worth right now?"
prompt = f"""
[SYSTEM]
{search_prompt}

[USER]
{query}

[ASSISTANT]
THOUGHT: 
"""
output = model(prompt, max_tokens=512, stop=["PAUSE"])["choices"][0]["text"]
search_query = None
for line in output.split("\n"):
    response_regex = "RESPONSE: (.*)"
    found_response = re.search(response_regex, line)
    if found_response:
        search_query = found_response.group(1)
        break
print(search_query)

Current net worth of Elon Musk


In [39]:
docs = []
if (search_query):
    params = {
        "key": GCP_API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query
    }
    res = requests.get(base_url, params)
    data = res.json()
    items = data["items"]
    i = 0
    while i < 3 and i < len(items):
        docs.append({"text": items[i]["snippet"], "source": items[i]["displayLink"]})
        i+=1
docs

[{'text': 'The wealth-tracking platform provides ongoing updates on the net worth and ranking of each individual confirmed by Forbes to be a billionaire ... Elon Musk. Tesla\xa0...',
  'source': 'www.forbes.com'},
 {'text': "May 30, 2024 ... Elon Musk's net worth was reported at $202 billion as of late May 2024, according to Bloomberg's Billionaire Index. His shares in Tesla and\xa0...",
  'source': 'www.thestreet.com'},
 {'text': 'Feb 7, 2024 ... Estimated net worth of $200 billion as of February 2024, making him the richest person in the world, according to Bloomberg. He is about $5\xa0...',
  'source': 'www.bankrate.com'}]

In [42]:
prompt = f"""
[SYSTEM]
{answer_prompt}

###CONTEXT:
{docs}

[USER]
{query}

[ASSISTANT]
"""
output = model(prompt, max_tokens=512, stop=["PAUSE"])["choices"][0]["text"]
output

'As of late May 2024, Elon Musk has a reported net worth of $202 billion according to the Bloomberg Billionaire Index. Prior to that, in February 2024, he was estimated to have a net worth of around $200 billion, making him the richest person in the world at that time, according to Bloomberg as well. These numbers are based on his shares in companies like Tesla and others.'

In [1]:
from search_agent import SearchAgent

c:\Users\jaide\Documents\CodingProjects\NeuroSymbolicAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
agent = SearchAgent()

In [3]:
output = agent.query("What is the current human population?", verbose=True)
print(output)

[{'text': 'How many people are there in the world? World population has reached 8 billion on November 15, 2022 according to the United Nations. World population live\xa0...', 'source': 'www.worldometers.info'}, {'text': 'World Population ; 1. China, 1,416,043,270, 6. Nigeria, 236,747,130 ; 2. India, 1,409,128,296, 7. Brazil, 220,051,512 ; 3. United States, 336,673,595, 8.', 'source': 'www.census.gov'}, {'text': '8,117,934,855Current World Population ; 64,994,135Births this year ; 260,613Births today ; 35,584,976Net population growth this year ; 12,126,606,410Public\xa0...', 'source': 'www.worldometers.info'}]
Current human population
According to the United Nations, as of November 15, 2022, there are around 8 billion people living on Earth. This number continues to grow with approximately 64,994,135 births expected this year and an estimated net population growth of about 35.5 million people for the year 2022. The countries with the highest population include China, India, Nigeria, Uni